In [1]:
pip install tensorflow


   ---------------------------------------- 0.0/422.5 kB ? eta -:--:--
    --------------------------------------- 10.2/422.5 kB ? eta -:--:--
    --------------------------------------- 10.2/422.5 kB ? eta -:--:--
   -- ------------------------------------ 30.7/422.5 kB 186.2 kB/s eta 0:00:03
   -- ------------------------------------ 30.7/422.5 kB 186.2 kB/s eta 0:00:03
   ----- --------------------------------- 61.4/422.5 kB 272.3 kB/s eta 0:00:02
   ----- --------------------------------- 61.4/422.5 kB 272.3 kB/s eta 0:00:02
   -------- ------------------------------ 92.2/422.5 kB 261.7 kB/s eta 0:00:02
   -------- ------------------------------ 92.2/422.5 kB 261.7 kB/s eta 0:00:02
   -------- ------------------------------ 92.2/422.5 kB 261.7 kB/s eta 0:00:02
   -------- ------------------------------ 92.2/422.5 kB 261.7 kB/s eta 0:00:02
   ----------- -------------------------- 122.9/422.5 kB 225.3 kB/s eta 0:00:02
   -------------- ----------------------- 163.8/422.5 kB 288.7 kB

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.65.4 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.23.4 which is incompatible.

[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, strides=stride, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    
    return x

def create_complex_cnn_model(input_shape=(224, 224, 3), num_classes=10):
    inputs = layers.Input(shape=input_shape)
    
    # Initial convolution
    x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
    
    # Residual blocks
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128)
    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 256)
    x = residual_block(x, 512, stride=2)
    x = residual_block(x, 512)
    
    # Global average pooling
    x = layers.GlobalAveragePooling2D()(x)
    
    # Dense layers
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    
    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    return model